# Automatic Gradient Computation - Reverse Mode

In [1]:
## Necessary Imports

import math
import random

from abc import ABC, abstractmethod

In [2]:
## Base class for the computation graph

class Tensor(ABC):
    def __add__(self, other):
        return self.add(self, self.scalar(other))
    
    def __sub__(self, other):
        return self.sub(self, self.scalar(other))
    
    def __mul__(self, other):
        return self.mul(self, self.scalar(other))
    
    def __truediv__(self, other):
        return self.div(self, self.scalar(other))
    
    def __pow__(self, other):
        return self.pow(self, self.scalar(other))

    def __radd__(self, other):
        return self.add(self.scalar(other), self)

    def __rsub__(self, other):
        return self.sub(self.scalar(other), self)

    def __rmul__(self, other):
        return self.mul(self.scalar(other), self)
    
    def __rtruediv__(self, other):
        return self.div(self.scalar(other), self)

    def __rpow__(self, other):
        return self.pow(self.scalar(other), self)
    
    @classmethod
    def scalar(cls, data, require_grad=False):
        if isinstance(data, Tensor):
            return data
        return Scalar(data, require_grad)
    
    @classmethod
    def sin(cls, op, require_grad=False):
        return Sine(op, require_grad)
    
    @classmethod
    def cos(cls, op, require_grad=False):
        return Cosine(op, require_grad)
    
    @classmethod
    def tanh(cls, op, require_grad=False):
        return Tanh(op, require_grad)
    
    @classmethod
    def exp(cls, op, require_grad=False):
        return Exponent(op, require_grad)
    
    @classmethod
    def add(cls, op1, op2, require_grad=False):
        return Add(op1, op2, require_grad)
    
    @classmethod
    def sub(cls, op1, op2, require_grad=False):
        return Subtract(op1, op2, require_grad)
    
    @classmethod
    def mul(cls, op1, op2, require_grad=False):
        return Multiply(op1, op2, require_grad)
    
    @classmethod
    def div(cls, op1, op2, require_grad=False):
        return Divide(op1, op2, require_grad)
    
    @classmethod
    def pow(cls, op1, op2, require_grad=False):
        return Power(op1, op2, require_grad)
    
    @abstractmethod
    def zero_grad(self):
        pass
    
    @abstractmethod
    def parameters(self):
        pass
    pass

In [3]:
## Class to hold scalar value

class Scalar(Tensor):
    def __init__(self, data, require_grad=False):
        self.data = data
        self.grad, self.require_grad = 0, require_grad
        pass

    def backward(self):
        return

    def zero_grad(self):
        self.grad = 0
        pass

    def parameters(self):
        yield self
    pass

In [4]:
## Parent class for Unary Operators

class UnaryOperator(Tensor):
    def __init__(self, op, require_grad=False):
        self.data = self.operate(op)
        self.grad, self.require_grad = 0, op.require_grad or require_grad

        self.op = op
        pass

    def backward(self):
        if self.require_grad:
            self.differentiate()
            self.op.backward()
        return

    def zero_grad(self):
        self.grad = 0
        self.op.zero_grad()
        pass

    def parameters(self):
        yield self
        yield from self.op.parameters()

    @classmethod
    @abstractmethod
    def operate(cls, op):
        pass

    @abstractmethod
    def differentiate(self):
        pass
    pass

In [5]:
## Unary Operators

class Sine(UnaryOperator):
    def __init__(self, op, require_grad=False):
        super().__init__(op, require_grad)
        pass

    @classmethod
    def operate(cls, op):
        return math.sin(op.data)

    def differentiate(self):
        self.op.grad += math.cos(self.data) * self.grad
        return
    pass


class Cosine(UnaryOperator):
    def __init__(self, op, require_grad=False):
        super().__init__(op, require_grad)
        pass

    @classmethod
    def operate(cls, op):
        return math.cos(op.data)

    def differentiate(self):
        self.op.grad -= math.sin(self.data) * self.grad
        return
    pass


class Tanh(UnaryOperator):
    def __init__(self, op, require_grad=False):
        super().__init__(op, require_grad)
        pass

    @classmethod
    def operate(cls, op):
        return math.tanh(op.data)

    def differentiate(self):
        self.op.grad += (1 - pow(math.tanh(self.data), 2)) * self.grad
        return
    pass


class Exponent(UnaryOperator):
    def __init__(self, op, require_grad=False):
        super().__init__(op, require_grad)
        pass

    @classmethod
    def operate(cls, op):
        return math.exp(op.data)

    def differentiate(self):
        self.op.grad += math.exp(self.data) * self.grad
        return
    pass

In [6]:
## Parent class for Binary Operators

class BinaryOperator(Tensor):
    def __init__(self, op1, op2, require_grad=False):
        self.data = self.operate(op1, op2)
        self.grad, self.require_grad = 0, op1.require_grad or op2.require_grad or require_grad

        self.op1, self.op2 = op1, op2
        pass

    def backward(self):
        if self.require_grad:
            self.differentiate()
            self.op1.backward()
            self.op2.backward()
        return

    def zero_grad(self):
        self.grad = 0
        self.op1.zero_grad()
        self.op2.zero_grad()
        pass

    def parameters(self):
        yield self
        yield from self.op1.parameters()
        yield from self.op2.parameters()

    @classmethod
    @abstractmethod
    def operate(cls, op1, op2):
        pass

    @abstractmethod
    def differentiate(self):
        pass
    pass

In [7]:
## Binary Operators

class Add(BinaryOperator):
    def __init__(self, op1, op2, require_grad=False):
        super().__init__(op1, op2, require_grad)
        pass

    @classmethod
    def operate(cls, op1, op2):
        return op1.data + op2.data

    def differentiate(self):
        self.op1.grad += self.grad
        self.op2.grad += self.grad
        return
    pass


class Subtract(BinaryOperator):
    def __init__(self, op1, op2, require_grad=False):
        super().__init__(op1, op2, require_grad)
        pass

    @classmethod
    def operate(cls, op1, op2):
        return op1.data - op2.data

    def differentiate(self):
        self.op1.grad += self.grad
        self.op2.grad -= self.grad
        return
    pass


class Multiply(BinaryOperator):
    def __init__(self, op1, op2, require_grad=False):
        super().__init__(op1, op2, require_grad)
        pass

    @classmethod
    def operate(cls, op1, op2):
        return op1.data * op2.data

    def differentiate(self):
        self.op1.grad += self.op2.data * self.grad
        self.op2.grad += self.op1.data * self.grad
        return
    pass


class Divide(BinaryOperator):
    def __init__(self, op1, op2, require_grad=False):
        super().__init__(op1, op2, require_grad)
        pass

    @classmethod
    def operate(cls, op1, op2):
        return op1.data / op2.data

    def differentiate(self):
        self.op1.grad += self.grad / self.op2.data
        self.op2.grad -= (self.op1.data / pow(self.op2.data, 2)) * self.grad
        return
    pass


class Power(BinaryOperator):
    def __init__(self, op1, op2, require_grad=False):
        super().__init__(op1, op2, require_grad)
        pass

    @classmethod
    def operate(cls, op1, op2):
        return pow(op1.data, op2.data)

    def differentiate(self):
        self.op1.grad += self.op2.data * pow(self.op1.data, self.op2.data-1) * self.grad

        if self.op1.data <= 0:
            self.op2.grad = float('-inf')
        else:
            self.op2.grad += pow(self.op1.data, self.op2.data) * math.log(self.op1.data) * self.grad
        return
    pass

In [8]:
## Ground Truths

M, C = random.randint(-10, 10), random.randint(-20, 20)
X = [ x for x in range(-20, 21) ]
Y = [ M*x + C for x in X ]

print(M, C)
print(X)
print(Y)

2 -9
[-20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[-49, -47, -45, -43, -41, -39, -37, -35, -33, -31, -29, -27, -25, -23, -21, -19, -17, -15, -13, -11, -9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]


In [9]:
## Weights and Biases

m, c = Tensor.scalar(random.random(), require_grad=True), Tensor.scalar(random.random(), require_grad=True)

In [10]:
## Initial weight and bias

print(m.data, c.data)

0.764340511813275 0.2950141610814745


In [11]:
## Training

lr = 0.001
epochs = 2000

for epoch in range(epochs):
    loss = 0
    for x, y in zip(X, Y):
        x, y  = Tensor.scalar(x), Tensor.scalar(y)
        yHat  = (m * x) + c
        loss += (y - yHat) ** 2
    
    # backward propogation
    loss /= len(X)
    loss.zero_grad()
    loss.grad = 1
    loss.backward()

    # gradient descent
    for param in loss.parameters():
        if not param.require_grad:
            continue
        # clip gradients, and then gradient descent
        param.data -= lr * max(-1, min(1, param.grad))
    
    # info per epoch
    print(f"{epoch}: {loss.data}")

0: 300.1559001591283
1: 285.44535907353605
2: 271.2088599879437
3: 257.44640290235145
4: 244.15798781675892
5: 231.34361473116684
6: 219.00328364557475
7: 207.13699455998363
8: 195.74474747439248
9: 184.82654238880122
10: 174.38237930320975
11: 164.41225821761833
12: 154.91617913202657
13: 145.89414204643492
14: 137.34614696084307
15: 129.27219387525116
16: 121.67228278965922
17: 114.54641370406704
18: 107.8945866184748
19: 101.71680153288244
20: 96.01305844729002
21: 90.78335736169747
22: 86.02769827610484
23: 81.74608119051206
24: 77.93850610491918
25: 74.60497301932622
26: 71.74548193373316
27: 69.36003284814001
28: 67.44862576254674
29: 66.01126067695338
30: 65.04793759135987
31: 64.5586565057663
32: 63.731999268942545
33: 63.24944218334905
34: 62.42950894652532
35: 61.95367586093179
36: 61.1404666241081
37: 60.6713575385146
38: 59.864872301690916
39: 59.40248721609737
40: 58.60272597927365
41: 58.14706489368013
42: 57.3540276568564
43: 56.9050905712629
44: 56.118777334439194
45: 5

In [12]:
## Weights and Biases, after training

print(m.data, c.data)

1.9943405118131659 -8.999999999999597


In [13]:
## Inference

for idx, (x, y) in enumerate(zip(X, Y)):
    print(f"{idx:2d}: {round((m.data * x) + c.data, 2):6.2f}, {y:5d}")

 0: -48.89,   -49
 1: -46.89,   -47
 2: -44.90,   -45
 3: -42.90,   -43
 4: -40.91,   -41
 5: -38.92,   -39
 6: -36.92,   -37
 7: -34.93,   -35
 8: -32.93,   -33
 9: -30.94,   -31
10: -28.94,   -29
11: -26.95,   -27
12: -24.95,   -25
13: -22.96,   -23
14: -20.97,   -21
15: -18.97,   -19
16: -16.98,   -17
17: -14.98,   -15
18: -12.99,   -13
19: -10.99,   -11
20:  -9.00,    -9
21:  -7.01,    -7
22:  -5.01,    -5
23:  -3.02,    -3
24:  -1.02,    -1
25:   0.97,     1
26:   2.97,     3
27:   4.96,     5
28:   6.95,     7
29:   8.95,     9
30:  10.94,    11
31:  12.94,    13
32:  14.93,    15
33:  16.93,    17
34:  18.92,    19
35:  20.92,    21
36:  22.91,    23
37:  24.90,    25
38:  26.90,    27
39:  28.89,    29
40:  30.89,    31
